<a href="https://colab.research.google.com/github/prawizard/TweetsClassification_NLP/blob/main/TweetEval/Sentiment_Analysis_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [19]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score
import requests
import collections

In [20]:

import os, string, collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import snowballstemmer
from nltk.corpus import stopwords

from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, load_model
from keras.layers.embeddings import Embedding
from keras.layers import Flatten, Dense, Dropout, Convolution1D, MaxPooling1D, SpatialDropout1D, Input 
from keras.layers import GlobalMaxPooling1D, concatenate, LSTM, Bidirectional
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D 
import numpy as np
from numpy import array
import pandas as pd
from sklearn.model_selection import train_test_split

# Constants

In [22]:
TRAIN_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_text.txt"
TRAIN_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_labels.txt"
VAL_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_text.txt"
VAL_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_labels.txt"
TEST_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_text.txt"
TEST_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_labels.txt"
VOCAB_SIZE=2000

# Access Data from the Files

## Download the .txt files to our runtime

In [23]:
r = requests.get(TRAIN_TEXT_URL, allow_redirects=True)
open('train_text.txt', 'wb').write(r.content)

r = requests.get(TRAIN_LABELS_URL, allow_redirects=True)
open('train_labels.txt', 'wb').write(r.content)

r = requests.get(VAL_TEXT_URL, allow_redirects=True)
open('val_text.txt', 'wb').write(r.content)

r = requests.get(VAL_LABELS_URL, allow_redirects=True)
open('val_labels.txt', 'wb').write(r.content)

r = requests.get(TEST_TEXT_URL, allow_redirects=True)
open('test_text.txt', 'wb').write(r.content)

r = requests.get(TEST_LABELS_URL, allow_redirects=True)
open('test_labels.txt', 'wb').write(r.content)

1720

## Read data from the files downloaded

In [24]:
# Tweets
stream=open("train_text.txt")
tweets=stream.readlines()
stream.close()
val_stream=open("val_text.txt")
val_tweets=val_stream.readlines()
val_stream.close()
test_stream=open("test_text.txt")
test_tweets=test_stream.readlines()
test_stream.close()

# Labels
stream=open("train_labels.txt")
tweetsLabels=stream.readlines()
stream.close()
val_stream=open("val_labels.txt")
val_tweetsLabels=val_stream.readlines()
val_stream.close()
test_stream=open("test_labels.txt")
test_tweetsLabels=test_stream.readlines()
test_stream.close()


# Labels
labels=[0]*len(tweetsLabels)
for i in range(len(tweetsLabels)):
  if tweetsLabels[i].find('\n')!=-1:
    labels[i]=int(re.sub('\n', '', tweetsLabels[i]))
val_labels=[0]*len(val_tweetsLabels)
for i in range(len(val_tweetsLabels)):
  if val_tweetsLabels[i].find('\n')!=-1:
    val_labels[i]=int(re.sub('\n', '', val_tweetsLabels[i]))
test_labels=[0]*len(test_tweetsLabels)
for i in range(len(test_tweetsLabels)):
  if test_tweetsLabels[i].find('\n')!=-1:
    test_labels[i]=int(re.sub('\n', '', test_tweetsLabels[i]))


In [25]:
print('Samples in Training set : ',len(labels),', Validation set : ', len(val_labels),', Test set : ', len(test_labels))

Samples in Training set :  11916 , Validation set :  1324 , Test set :  860


# Pre-processing of the text in tweets

## Remove the '@User' text parts from tweets


In [26]:

for i in range(len(tweets)):
  if tweets[i].find('@user')!=-1:
    tweets[i]=re.sub('@user', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('@user')!=-1:
    val_tweets[i]=re.sub('@user', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('@user')!=-1:
    test_tweets[i]=re.sub('@user', '', test_tweets[i])

## Remove hashtags

In [27]:
for i in range(len(tweets)):
  if tweets[i].find('#[a-zA-Z]+')!=-1:
    tweets[i]=re.sub('#[a-zA-Z]+', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('#[a-zA-Z]+')!=-1:
    val_tweets[i]=re.sub('#[a-zA-Z]+', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('#[a-zA-Z]+')!=-1:
    test_tweets[i]=re.sub('#[a-zA-Z]+', '', test_tweets[i])

## Remove escape characters, unnecessary characters if present and correct wrongly spelt words

In [28]:
for i in range(len(tweets)):
  if tweets[i].find('\n')!=-1:
    tweets[i]=re.sub('\n', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('\n')!=-1:
    val_tweets[i]=re.sub('\n', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('\n')!=-1:
    test_tweets[i]=re.sub('\n', '', test_tweets[i])

# Unnecessary dots
p='\.\.\.|\.\.'

for i in range(len(tweets)):
  tweets[i]=re.sub(p, '', tweets[i])
  tweets[i]=tweets[i].lower()

for i in range(len(val_tweets)):
  val_tweets[i]=re.sub(p, '', val_tweets[i])
  val_tweets[i]=val_tweets[i].lower()

for i in range(len(test_tweets)):
  test_tweets[i]=re.sub(p, '', test_tweets[i])
  test_tweets[i]=test_tweets[i].lower()

## Store hashtags to replace after tokenization

In [29]:
h='#[a-zA-Z]+'
hashdict={}
for i in range(len(tweets)):
  hashtags=re.findall(h, tweets[i])
  for w in hashtags:
    hashdict[w]=w[1:]

for i in range(len(val_tweets)):
  hashtags=re.findall(h, val_tweets[i])
  for w in hashtags:
    hashdict[w]=w[1:]

for i in range(len(test_tweets)):
  hashtags=re.findall(h, test_tweets[i])
  for w in hashtags:
    hashdict[w]=w[1:]

# Make a data frame for the dataset

In [30]:
rows=[]
rowIndices=[]
for i in range(len(tweets)):
  rows.append({"TWEET":tweets[i], "CATEGORY":labels[i]})
  rowIndices.append(i+1)
df=pd.DataFrame(rows, index=rowIndices)

val_rows=[]
val_rowIndices=[]
for i in range(len(val_tweets)):
  val_rows.append({"TWEET":val_tweets[i], "CATEGORY":val_labels[i]})
  val_rowIndices.append(i+1)
val_df=pd.DataFrame(val_rows, index=val_rowIndices)

test_rows=[]
test_rowIndices=[]
for i in range(len(test_tweets)):
  test_rows.append({"TWEET":test_tweets[i], "CATEGORY":test_labels[i]})
  test_rowIndices.append(i+1)
test_df=pd.DataFrame(test_rows, index=test_rowIndices)

In [31]:
df.head()

,TWEET,CATEGORY
1,bono who cares. soon people will understand t...,0
2,eight years the republicans denied obama’s pi...,1
3,get him some line help. he is gonna be just f...,0
4,she is great. hi fiona!,0
5,she has become a parody unto herself? she has...,1


In [32]:
val_df.head()

,TWEET,CATEGORY
1,wiiu is not even a real console.,0
2,if he is from az i would put my money on se...,1
3,i thought canada had strict gun control. hel...,0
4,following all #maga patriots p...,0
5,1 minute of truth: gun control via,0


In [33]:
test_df.head()

,TWEET,CATEGORY
1,#ibelieveblaseyford is liar she is fat ugly li...,1
2,i got in a pretty deep debate with my frien...,0
3,"if you want more shootings and more death, the...",0
4,angels now have 6 runs. five of them have come...,0
5,#travel #movies and unix #fortune combined vi...,0


# Download stopwords

In [34]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Tokenize the tweets

In [35]:
rows=[]
rowIndices=[]
df_tweets=np.array(df.TWEET)
sym=['#','$','?','@','%','^','!','&',',','.','/',"'",'`','``',"''",'(',')']
for i in range(len(df_tweets)):
  tokens=word_tokenize(df_tweets[i])
  for x in range(len(tokens)):
    if tokens[x]=='ca':
      tokens[x]='can'
    if tokens[x]=="n't":
      tokens[x]='not'
  tokens=[token for token in tokens if token not in sym]
  rows.append({"TWEET":tokens, "CATEGORY":labels[i]})
  rowIndices.append(i+1)
tokenized_df=pd.DataFrame(rows, index=rowIndices)

val_rows=[]
val_rowIndices=[]
val_df_tweets=np.array(val_df.TWEET)
for i in range(len(val_df_tweets)):
  val_tokens=word_tokenize(val_df_tweets[i])
  for x in range(len(val_tokens)):
    if val_tokens[x]=='ca':
      val_tokens[x]='can'
    if val_tokens[x]=="n't":
      val_tokens[x]='not'
  val_tokens=[token for token in val_tokens if token not in sym]
  val_rows.append({"TWEET":val_tokens, "CATEGORY":val_labels[i]})
  val_rowIndices.append(i+1)
tokenized_val_df=pd.DataFrame(val_rows, index=val_rowIndices)

test_rows=[]
test_rowIndices=[]
test_df_tweets=np.array(test_df.TWEET)
for i in range(len(test_df_tweets)):
  test_tokens=word_tokenize(test_df_tweets[i])
  for x in range(len(test_tokens)):
    if test_tokens[x]=='ca':
      test_tokens[x]='can'
    if test_tokens[x]=="n't":
      test_tokens[x]='not'
  test_tokens=[token for token in test_tokens if token not in sym]
  test_rows.append({"TWEET":test_tokens, "CATEGORY":test_labels[i]})
  test_rowIndices.append(i+1)
tokenized_test_df=pd.DataFrame(test_rows, index=test_rowIndices)

In [36]:
tokenized_df.head()

,TWEET,CATEGORY
1,"[bono, who, cares, soon, people, will, underst...",0
2,"[eight, years, the, republicans, denied, obama...",1
3,"[get, him, some, line, help, he, is, gon, na, ...",0
4,"[she, is, great, hi, fiona]",0
5,"[she, has, become, a, parody, unto, herself, s...",1


In [37]:
tokenized_val_df.head()

,TWEET,CATEGORY
1,"[wiiu, is, not, even, a, real, console]",0
2,"[if, he, is, from, az, i, would, put, my, mone...",1
3,"[i, thought, canada, had, strict, gun, control...",0
4,"[following, all, maga, patriots, please, follo...",0
5,"[1, minute, of, truth, :, gun, control, via]",0


In [38]:
tokenized_test_df.head()

,TWEET,CATEGORY
1,"[ibelieveblaseyford, is, liar, she, is, fat, u...",1
2,"[i, got, in, a, pretty, deep, debate, with, my...",0
3,"[if, you, want, more, shootings, and, more, de...",0
4,"[angels, now, have, 6, runs, five, of, them, h...",0
5,"[travel, movies, and, unix, fortune, combined,...",0


In [39]:
def update_vocab_counter(row):
    for word in row:
        vocab_counter[word] += 1

In [40]:
vocab_counter = collections.Counter()
tokenized_df['TWEET'].apply(update_vocab_counter);
vocab = sorted(vocab_counter, key=vocab_counter.get, reverse=True)

In [41]:
len(vocab)

20833

In [42]:
max_words = 5000

In [43]:
w2id = {w:i for i, w in enumerate(vocab[:max_words])}
w2id['unk'] = -1

In [44]:
def transform_to_ids(row):
    return [w2id[w] if w in w2id else w2id['unk'] for w in row]

In [45]:
tokenized_df['tokenized_int'] = tokenized_df['TWEET'].apply(lambda x: transform_to_ids(x))

In [46]:
tokenized_val_df['tokenized_int'] = tokenized_val_df['TWEET'].apply(lambda x: transform_to_ids(x))
tokenized_test_df['tokenized_int'] = tokenized_test_df['TWEET'].apply(lambda x: transform_to_ids(x))

In [47]:
tokenized_df.head()

,TWEET,CATEGORY,tokenized_int
1,"[bono, who, cares, soon, people, will, underst...",0,"[1225, 45, 683, 568, 49, 47, 312, 10, 19, 1813..."
2,"[eight, years, the, republicans, denied, obama...",1,"[4712, 159, 0, 286, 4713, 252, 14, 37, 3337, -..."
3,"[get, him, some, line, help, he, is, gon, na, ...",0,"[66, 74, 114, 471, 195, 9, 1, 384, 269, 22, 42..."
4,"[she, is, great, hi, fiona]",0,"[15, 1, 152, 1344, 4714]"
5,"[she, has, become, a, parody, unto, herself, s...",1,"[15, 59, 413, 4, -1, -1, 728, 15, 59, 1345, 94..."


In [48]:
lens = tokenized_df['tokenized_int'].apply(lambda x: len(x))
min(lens), max(lens), np.mean(lens)

(1, 69, 20.31117824773414)

In [50]:
a=tokenized_df['tokenized_int'].values
j=0
for i in a:
  if len(i)==69:
    z=i
    k=j
  j+=1
print(z)
print(tokenized_df['TWEET'].values[k])

[7, 14, 169, 237, 5, 31, 3, 8, 467, 65, 1, 10, 7, 46, 22, 12, 57, 224, -1, 5, 3, 46, 42, 1061, 12, 5, 538, 65, 51, 10, 14, 37, 11, 13, 139, 14, 37, 504, 4, 1377, 781, 104, 360, 7, 14, 169, 0, 495, 17, 3622, 7, 298, 59, 2, 22, 3028, 153, 21, 107, 35, 7, 47, 22, 54, 0, 272, 1226, 7, 1832]
['i', '’', 'm', 'black', 'and', 'what', 'you', 'are', 'telling', 'me', 'is', 'that', 'i', 'can', 'be', 'in', 'my', 'own', 'dwelling', 'and', 'you', 'can', 'just', 'walk', 'in', 'and', 'kill', 'me', 'no', 'that', '’', 's', 'not', 'it', 'let', '’', 's', 'run', 'a', 'smear', 'campaign', 'even', 'though', 'i', '’', 'm', 'the', 'victim', 'this', 'rage', 'i', 'feel', 'has', 'to', 'be', 'settled', 'down', 'with', 'love', 'but', 'i', 'will', 'be', 'at', 'the', 'next', 'rally', 'i', 'promise']


In [149]:
maxlen = 70

In [150]:
X_train=tokenized_df['tokenized_int'].values
X_test=tokenized_test_df['tokenized_int'].values
y_train=tokenized_df['CATEGORY'].values
y_test=tokenized_test_df['CATEGORY'].values

In [151]:
x_train = pad_sequences(X_train, maxlen=maxlen, value=-1)
x_test = pad_sequences(X_test, maxlen=maxlen, value=-1)

In [152]:
dummy_y = np_utils.to_categorical(y_train)
dummy_y_test = np_utils.to_categorical(y_test)

In [153]:
def baseline_model():
    model = Sequential([Embedding(input_dim=max_words, output_dim=50, input_length=maxlen),
                        Flatten(),
                        Dense(2, activation='softmax')])

    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    return model

In [154]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=5, batch_size=100, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)